In [10]:
from tkinter import *
import tkinter.messagebox
from tkinter import ttk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd

root = Tk()
root.geometry("700x500")#定义总布局大小
root.resizable(width=False, height=False)#界面大小不可调整
root.title('网桥转发表模拟工具')
#图片部分框架
lbPic=LabelFrame(root, width=700, height=250)
lbPic.grid(row=0,column=0)
#发送部分框架
sendfm = Frame(root, width = 700, height=40)
sendfm.grid(row=1,column=0)
#转发表表头
lbhead = LabelFrame(root, width = 700, height = 50, padx=50)
lbhead.grid(row=2,column=0)
#转发表数据
lbdata = Frame(root, width = 700, height = 160, padx=50)
lbdata.grid(row=3,column=0)
#主机所在局域网
lan1 = ["H1","H2"]
lan2 = ["H3","H4","H5"]
lan3 = ["H6","H7"]
#主机端口记录
hostSegOne = {"H1":1,"H2":2,"H3":3,"H4":3, "H5":3, "H6":3, "H7":3}
hostSegTwo = {"H1":1,"H2":1,"H3":2,"H4":3,"H5":4, "H6":5, "H7":5}
hostSegThree = {"H1":1,"H2":1,"H3":1,"H4":1,"H5":1,"H6":2,"H7":3}
#转发表
S1 = {}
S2 = {}
S3 = {}
dataIO = []
srcAddr = ""
desAddr = ""
flag = False#默认为点击读取，
i = 0
# 下拉菜单的选择事件
# def chooseSrc(*args):
#     print(cbSrcAddr.get())
#读取数据
def tableRead():
    #root.withdraw()
    global flag,srcAddr,desAddr
    flag = True
    file_path = filedialog.askopenfilename(filetypes = [('CSV', 'csv')])
    info=pd.read_csv(file_path,encoding = 'gbk')
    for i in info['发送的帧']:
        srcAddr = i.split('-')[0]
        desAddr = i.split('>')[1]
        btnSend()
#         for i in obj:
#             print(i)
    #except:
        #tkinter.messagebox.showinfo('提示', '文件不存在，请先保存')
# def hello():
#     print("c");

#关于作者
def aboutAuthor():
    tkinter.messagebox.showinfo('作者及分工', '张宇：逆向学习算法的实现，程序总体功能设计\n王含艺：python的Tkinter库交互式界面布局\n吴昊航：程序框架、算法流程图')
    
#保存数据
def tableSave():
    global dataIO
    boolSave=tkinter.messagebox.askokcancel('提示', '要是否进行保存?')
    if(boolSave):
        colName = ['发送的帧','S1地址','S1接口','S2地址','S2接口','S3地址','S3接口','S1的处理','S2的处理','S3的处理']
        csvFile = pd.DataFrame(columns = colName, data = dataIO)
        csvFile.to_csv('test.csv',encoding='gbk',index_label = "序号")
    dataIO = []
#重置事件
def btnReset():
    #lbdata.destory()
    global i
    i = 0
    for widget in lbdata.winfo_children():
        widget.destroy()
    S1.clear()
    S2.clear()
    S3.clear()
    dataIO = []
#发送事件
def btnSend():
    global i,srcAddr,desAddr
    if(i > 7):
        tkinter.messagebox.showinfo('提示','转发信息已到达最大数据')
        return
#         boolSave=tkinter.messagebox.askokcancel('提示', '要是否进行保存?')
#         if(boolSave):
#             tableSave(dataIO)
#         return
    dataChip = ["","","","","","","","","",""]
    if(not flag):
        srcAddr = varSrc.get()
        desAddr = varDes.get()
    if(srcAddr == desAddr):
        return
    handleB1 = ""
    handleB2 = ""
    handleB3 = ""
    seqS = [0,0,0]
    loc = 0
    if(srcAddr in lan1):
        seqS = [1,2,3]
    elif(srcAddr in lan2):
        seqS = [2,1,3]
    elif(srcAddr in lan3):
        seqS = [3,2,1]
    else:
        return
    if(desAddr not in lan1):
        if(desAddr not in lan2):
            if(desAddr not in lan3):
                return
    count = 0
    while(count!=3):
        if(seqS[count] == 1):
            if(srcAddr in S1):
                handleB1 = "转发,不登记"
            else:
                S1[srcAddr] = hostSegOne[srcAddr]
                handleB1 = "转发,登记"
                Label(lbdata, text=srcAddr).place(x=65,y=20*i)
                Label(lbdata, text=S1[srcAddr]).place(x=125,y=20*i)
                dataChip[2 * count + 1] = srcAddr
                dataChip[2 * count + 2] = S1[srcAddr]
            if(desAddr in S1):
                if(desAddr in lan1):
                    for c in range(count + 1, 3):
                        seqS[c] = 0
                    
        elif(seqS[count] == 2):
            if(srcAddr in S2):
                handleB2 = "转发,不登记"
            else:
                S2[srcAddr] = hostSegTwo[srcAddr]
                handleB2 = "转发,登记"
                Label(lbdata, text=srcAddr).place(x=175,y=20*i)
                Label(lbdata, text=S2[srcAddr]).place(x=235,y=20*i)
                dataChip[2 * count + 1] = srcAddr
                dataChip[2 * count + 2] = S2[srcAddr]
            if(desAddr in S2):
                q = 2
                if(desAddr in lan1):
                    q = 3
                elif(desAddr in lan3):
                    q = 1
                for c in range(count, 3):
                    if(seqS[c] == q):
                        seqS[c] = 0
                    if(q == 2):
                        seqS[c] = 0
            #if(desAddr in S2):
        elif(seqS[count] == 3):
            if(srcAddr in S3):
                handleB3 = "转发,不登记"
            else:
                S3[srcAddr] = hostSegThree[srcAddr]
                handleB3 = "转发,登记"
                Label(lbdata, text=srcAddr).place(x=285,y=20*i)
                Label(lbdata, text=S3[srcAddr]).place(x=335,y=20*i)
                dataChip[2 * count + 1] = srcAddr
                dataChip[2 * count + 2] = S3[srcAddr]
            if(desAddr in S3):
                if(desAddr in lan3):
                    for c in range(count + 1, 3):
                        seqS[c] = 0
        count = count + 1
    Label(lbdata, text=srcAddr + "->" + desAddr).place(x=0,y=20*i)
    Label(lbdata, text=handleB1).place(x=380,y=20*i)
    Label(lbdata, text=handleB2).place(x=450,y=20*i)
    Label(lbdata, text=handleB3).place(x=520,y=20*i)
    dataChip[0] = srcAddr + "->" + desAddr
    dataChip[7] = handleB1
    dataChip[8] = handleB2
    dataChip[9] = handleB3
    dataIO.append(dataChip)
    i = i + 1
#     if(i == 7):
#         i = i - 1
#         boolSave=tkinter.messagebox.askokcancel('提示', '要是否进行保存?')
#         if(boolSave):
#             tableSave(dataIO)
        

if __name__ == "__main__":
    varSrc = StringVar()
    varDes = StringVar()
    img = Image.open('VLN1.png')  # 打开图片
    photo = ImageTk.PhotoImage(img)  # 用PIL模块的PhotoImage打开
    imglabel = Label(lbPic, image=photo)
    imglabel.place(x=0,y=0)
    #顶部菜单栏
    menubar = Menu(root)
    menubar.add_command(label="读取文件",command=tableRead)
    menubar.add_command(label="保存",command=tableSave)
    menubar.add_command(label="重置",command=btnReset)
    menubar.add_command(label="关于",command=aboutAuthor)
    root['menu']=menubar
    #出发地址Label
    lbStart = Label(sendfm, text= "出发地址:")
    lbStart.place(x=30,y=10)
    #出发地址下拉菜单
    cbSrcAddr=ttk.Combobox(sendfm,textvariable=varSrc) #初始化 
    cbSrcAddr["values"]=("H1","H2","H3","H4","H5","H6","H7")
    cbSrcAddr.current(0)  #选择第一个  
    cbSrcAddr.bind("<<ComboboxSelected>>")  #下拉列表框被选中事件
    cbSrcAddr.place(x=100,y=10)
    #目的地址Label
    lbEnd = Label(sendfm, text= "目的地址:")#.grid(row=7, column = 7)
    lbEnd.place(x=330,y=10)
    #目的地址下拉菜单
    cbDesAddr=ttk.Combobox(sendfm,textvariable=varDes) #初始化 
    cbDesAddr["values"]=("H1","H2","H3","H4","H5","H6","H7")
    cbDesAddr.current(0)  #选择第一个  
    cbDesAddr.bind("<<ComboboxSelected>>")  #下拉列表框被选中事件
    cbDesAddr.place(x=400,y=10)
    #发送按钮
    btn = Button(sendfm,text="发送",command=btnSend)
    btn.place(x=620,y=5)
    #重置按钮
    #bt = Button(sendfm,text="重置",command=btnReset)
    #bt.place(x=650,y=5)
    #表头
    Label(lbhead, text="发送的帧").place(x=0,y=12)
    Label(lbhead, text="S1的转发表").place(x=70,y=2)
    Label(lbhead, text="地址").place(x=60,y=20)
    Label(lbhead, text="接口").place(x=120,y=20)
    Label(lbhead, text="S2的转发表").place(x=180,y=2)
    Label(lbhead, text="地址").place(x=170,y=20)
    Label(lbhead, text="接口").place(x=230,y=20)
    Label(lbhead, text="S3的转发表").place(x=290,y=2)
    Label(lbhead, text="地址").place(x=280,y=20)
    Label(lbhead, text="接口").place(x=330,y=20)
    Label(lbhead, text="S1的处理").place(x=380,y=12)
    Label(lbhead, text="S2的处理").place(x=450,y=12)
    Label(lbhead, text="S3的处理").place(x=520,y=12)
    mainloop()


In [ ]:
#!/usr/bin/env python
# _*_ coding:utf-8 _*_
from tkinter import *
#定义一个顶级大窗口
root = Tk()
#在大窗口下定义一个菜单实例
menubar = Menu(root)
#给菜单实例增加菜单项
for each in ['读取文件','保存','重置','关于']:
    menubar.add_command(label=each, , command=hello)
    

#菜单实例应用到大窗口中
menubar.add_command(label=each, , command=hello)
root['menu']=menubar
root.mainloop()

In [ ]:
#!/usr/bin/env python
# _*_ coding:utf-8 _*_
from tkinter import *
#定义一个顶级大窗口
root = Tk()
#在大窗口下定义一个顶级菜单实例
menubar = Menu(root)
 
#在顶级菜单实例下创建子菜单实例
fmenu = Menu(menubar)
for each in ['新建','打开','保存','另存为']:
    fmenu.add_command(label=each)
vmenu = Menu(menubar)
#为每个子菜单实例添加菜单项
for each in ['复制','粘贴','剪切']:
    vmenu.add_command(label=each)
emenu = Menu(menubar)
for each in ['默认视图','新式视图']:
    emenu.add_command(label=each)
amenu = Menu(menubar)
for each in ['版权信息','联系我们']:
    amenu.add_command(label=each)
 
 #为顶级菜单实例添加菜单，并级联相应的子菜单实例
menubar.add_cascade(label='文件',menu=fmenu)
menubar.add_cascade(label='视图',menu=vmenu)
menubar.add_cascade(label='编辑',menu=emenu)
menubar.add_cascade(label='关于',menu=amenu)
 
#顶级菜单实例应用到大窗口中
root['menu']=menubar
root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import filedialog
 
root = tk.Tk()
#root.withdraw()
 
file_path = filedialog.askopenfilename()
 
print(file_path)
